# TS1 Current Moderator - New Split

This is the Hydrogen gas and polymer production for the current TS1 moderator that was split into 48 sections as shown in the "current TS1, split methane forOT" powerpoint. 
- It includes results for eight different particles
- Also includes implemented dwell time estimates
    * The estimates were made based upon the diagrams in the powerpoint above and assigned to the corresponding split sections
- Functions in the 'Polymer_Production_Functions.py' script based upon the maths done in (Evans 1995)
- All particles of sufficient energy can induce bond cleavage which leads to the polymerisation reaction so it can be assumed that all particles present contribute to the polymer production.
    - The amount of spurs/reactive species created can differ depending on whether the particle is High LET or Low LET. High LET can create spurs in a much closer proximity to eachother which can lead to overlap and therefore inter-spur reactions as well as other phenomena such as delta rays (more in radiolysis draft) 
    
Evans paper : Evans, D., 1995. Irradiation effects in liquid methane used as a neutron moderator. Cryogenics, 35(11), pp.763-766.


In [110]:
import numpy as np
import pandas as pd

from Polymer_Production_Functions import Mod_Data_Reader
from Polymer_Production_Functions import moles_per_MGy_per_g
from Polymer_Production_Functions import radicals_per_MGy_per_g
from Polymer_Production_Functions import vol_of_H_per_s
from Polymer_Production_Functions import rate_of_pol_form
from Polymer_Production_Functions import molecular_weight
from Polymer_Production_Functions import MeV_per_g_per_sec_to_MGy_per_sec

## Moles/MGy/g of CH4, No of Radicals/MGy/g of CH4 and Molecular weight
- For the hydrogen gas and the polymer 
- The molecular weight is for the assumption that the chain length is n = 20 (thought to be the maximum) and of the form (CH2)n (Evans)
- G-Values for CH4 in umol/J, taken from (Evans 1995).
    - Radiochemical yield from methane

In [111]:
moles_H_per_MGyg = moles_per_MGy_per_g(0.66e-6)
print (moles_H_per_MGyg, 'moles of Hydrogen gas per MGy per gram of CH4')
print (radicals_per_MGy_per_g(moles_H_per_MGyg), 'radicals/g.MGy of Hydrogen')

moles_Pol_per_MGyg = moles_per_MGy_per_g(0.22e-6)
print (moles_Pol_per_MGyg, 'moles of Polymer per MGy per gram of CH4')
print (radicals_per_MGy_per_g(moles_Pol_per_MGyg), 'radicals/g.MGy of Polymer')

Mr_20 = molecular_weight(20)
print('Molecular weight of the polymer (CH2)n for the max chain length of 20 =', Mr_20)

0.00066 moles of Hydrogen gas per MGy per gram of CH4
3.97518e+20 radicals/g.MGy of Hydrogen
0.00022 moles of Polymer per MGy per gram of CH4
1.32506e+20 radicals/g.MGy of Polymer
Molecular weight of the polymer (CH2)n for the max chain length of 20 = 280


## Individual Particle Data DataFrames
Reading in the files to dfs and calculating energy deposited per gram and dose rate for each particle

In [112]:
#paths
n_path = 'Data_for_OT/neutrons_heating_current_ts1.txt'
a_path = 'Data_for_OT/alphas_heating_current_ts1.txt'
d_path = 'Data_for_OT/deuterons_heating_current_ts1.txt'
h_path = 'Data_for_OT/helions_heating_current_ts1.txt'
t_path = 'Data_for_OT/tritons_heating_current_ts1.txt'
p_path = 'Data_for_OT/protons_heating_current_ts1.txt'
ph_path = 'Data_for_OT/photons_heating_current_ts1.txt'
pi_path = 'Data_for_OT/pi_plus_heating_current_ts1.txt'

In [113]:
#reading to dfs
n_df = Mod_Data_Reader(n_path)
a_df = Mod_Data_Reader(a_path)
d_df = Mod_Data_Reader(d_path)
h_df = Mod_Data_Reader(h_path)
t_df = Mod_Data_Reader(t_path)
p_df = Mod_Data_Reader(p_path)
ph_df = Mod_Data_Reader(ph_path)
pi_df = Mod_Data_Reader(pi_path)

In [114]:
#calculating energy deposited per gram and absorbed dose rate

#proton beam current
pps = 1.00E+15

E_dep_n = (n_df['raw_energy_MeV']/n_df['mass g'])*pps
E_dep_a = (a_df['raw_energy_MeV']/a_df['mass g'])*pps
E_dep_d = (d_df['raw_energy_MeV']/d_df['mass g'])*pps
E_dep_h = (h_df['raw_energy_MeV']/h_df['mass g'])*pps
E_dep_t = (t_df['raw_energy_MeV']/t_df['mass g'])*pps
E_dep_p = (p_df['raw_energy_MeV']/p_df['mass g'])*pps
E_dep_ph = (ph_df['raw_energy_MeV']/ph_df['mass g'])*pps
E_dep_pi = (pi_df['raw_energy_MeV']/pi_df['mass g'])*pps

dose_rate_n = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_n)
dose_rate_a = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_a)
dose_rate_d = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_d)
dose_rate_h = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_h)
dose_rate_t = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_t)
dose_rate_p = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_p)
dose_rate_ph = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_ph)
dose_rate_pi = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_pi)

## All Particles Dose Rates DataFrame
Appending all particles dose rates as well as CH4 mass and volume to one dataframe

In [115]:
All_DR_df = pd.DataFrame(columns = ['Cell #', 
                                            'CH4 Vol cc',
                                            'CH4 Mass g',
                                            'Neutron Dose Rate MGy/s',
                                            'Alpha Dose Rate MGy/s',
                                            'Deuteron Dose Rate MGy/s',
                                            'Helion Dose Rate MGy/s',
                                            'Triton Dose Rate MGy/s',
                                            'Proton Dose Rate MGy/s',
                                            'Photon Dose Rate MGy/s',
                                            'Pion + Dose Rate MGy/s'
                                           ])
All_DR_df['Cell #'] = n_df['cell num']
All_DR_df['CH4 Vol cc'] = n_df['volume cc']
All_DR_df['CH4 Mass g'] = n_df['mass g']
All_DR_df['Neutron Dose Rate MGy/s'] = dose_rate_n
All_DR_df['Alpha Dose Rate MGy/s'] = dose_rate_a
All_DR_df['Deuteron Dose Rate MGy/s'] = dose_rate_d
All_DR_df['Helion Dose Rate MGy/s'] = dose_rate_h
All_DR_df['Triton Dose Rate MGy/s'] = dose_rate_t
All_DR_df['Proton Dose Rate MGy/s'] = dose_rate_p
All_DR_df['Photon Dose Rate MGy/s'] = dose_rate_ph
All_DR_df['Pion + Dose Rate MGy/s'] = dose_rate_pi

#All_DR_df

## Calculating Hydrogen and Polymer Production Rates for All Particles
- Using (Evans 1995) value of 0.43 for specific gravity - essentially acts as the density in cc/g
- Polymer G value from (Evans 1995) (umol/J)
- Using molecular weight for n=20

In [116]:
#Volume of H gas rate
H_rate_n = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Neutron Dose Rate MGy/s'], 0.43)
H_rate_a = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Alpha Dose Rate MGy/s'], 0.43)
H_rate_d = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Deuteron Dose Rate MGy/s'], 0.43)
H_rate_h = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Helion Dose Rate MGy/s'], 0.43)
H_rate_t = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Triton Dose Rate MGy/s'], 0.43)
H_rate_p = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Proton Dose Rate MGy/s'], 0.43)
H_rate_ph = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Photon Dose Rate MGy/s'], 0.43)
H_rate_pi = vol_of_H_per_s(moles_H_per_MGyg, All_DR_df['CH4 Vol cc'], All_DR_df['Pion + Dose Rate MGy/s'], 0.43)

In [117]:
#Polymer production rate
Pol_rate_n = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Neutron Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_a = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Alpha Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_d = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Deuteron Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_h = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Helion Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_t = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Triton Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_p = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Proton Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_ph = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Photon Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])
Pol_rate_pi = rate_of_pol_form(0.22e-6, Mr_20, All_DR_df['Pion + Dose Rate MGy/s'], All_DR_df['CH4 Mass g'])

## All Particle Results DataFrames
Dataframe for H gas production rate and polymer production rate

In [118]:
H_rates_df = pd.DataFrame(columns = ['Cell #',
                                            'Neutron Hydrogen Rate cc/s',
                                            'Alpha Hydrogen Rate cc/s',
                                            'Deuteron Hydrogen Rate cc/s',
                                            'Helion Hydrogen Rate cc/s',
                                            'Triton Hydrogen Rate cc/s',
                                            'Proton Hydrogen Rate cc/s',
                                            'Photon Hydrogen Rate cc/s',
                                            'Pion + Hydrogen Rate cc/s'
                                            ])

Pol_rates_df = pd.DataFrame(columns = ['Cell #', 
                                              'Neutron Polymer Rate g/s',
                                              'Alpha Polymer Rate g/s',
                                              'Deuteron Polymer Rate g/s',
                                              'Helion Polymer Rate g/s',
                                              'Triton Polymer Rate g/s',
                                              'Proton Polymer Rate g/s',
                                              'Photon Polymer Rate g/s',
                                              'Pion + Polymer Rate g/s'
                                              ])


In [119]:
#Appending the values to H df
H_rates_df['Cell #'] = n_df['cell num']
H_rates_df['Neutron Hydrogen Rate cc/s'] = H_rate_n
H_rates_df['Alpha Hydrogen Rate cc/s'] = H_rate_a
H_rates_df['Deuteron Hydrogen Rate cc/s'] = H_rate_d
H_rates_df['Helion Hydrogen Rate cc/s'] = H_rate_h
H_rates_df['Triton Hydrogen Rate cc/s'] = H_rate_t
H_rates_df['Proton Hydrogen Rate cc/s'] = H_rate_p
H_rates_df['Photon Hydrogen Rate cc/s'] = H_rate_ph
H_rates_df['Pion + Hydrogen Rate cc/s'] = H_rate_pi

H_rates_df

Cell #  Neutron Hydrogen Rate cc/s  Alpha Hydrogen Rate cc/s  \
0       13                    0.027770                  0.000283   
1       14                    0.024824                  0.000235   
2    13000                    0.019308                  0.000265   
3    14000                    0.016702                  0.000255   
4    13001                    0.011906                  0.000173   
5    14001                    0.010340                  0.000162   
6    13002                    0.006566                  0.000077   
7    14002                    0.005854                  0.000085   
8    13003                    0.032848                  0.000388   
9    14003                    0.029155                  0.000294   
10   13004                    0.021058                  0.000297   
11   14004                    0.017977                  0.000260   
12   13005                    0.013099                  0.000167   
13   14005                    0.010977                  0.000148   
14   13006                    0.007809                  0.000092   
15   14006                    0.006698                  0.000123   
16   13007                    0.032550                  0.000359   
17   14007                    0.029099                  0.000212   
18   13008                    0.020524                  0.000235   
19   14008                    0.017626                  0.000204   
20   13009                    0.012929                  0.000255   
21   14009                    0.010888                  0.000151   
22   13010                    0.008014                  0.000131   
23   14010                    0.006876                  0.000116   
24   13011                    0.032013                  0.000388   
25   14011                    0.029263                  0.000251   
26   13012                    0.020788                  0.000231   
27   14012                    0.018217                  0.000246   
28   13013                    0.013478                  0.000163   
29   14013                    0.011540                  0.000180   
30   13014                    0.008468                  0.000087   
31   14014                    0.007401                  0.000101   
32   13015                    0.019901                  0.000152   
33   14015                    0.018644                  0.000118   
34   13016                    0.013994                  0.000186   
35   14016                    0.012556                  0.000144   
36   13017                    0.009393                  0.000148   
37   14017                    0.008278                  0.000097   
38   13018                    0.005985                  0.000055   
39   14018                    0.005260                  0.000068   
40   13019                    0.011270                  0.000085   
41   14019                    0.010844                  0.000086   
42   13020                    0.008961                  0.000048   
43   14020                    0.008236                  0.000069   
44   13021                    0.006242                  0.000070   
45   14021                    0.005693                  0.000059   
46   13022                    0.003813                  0.000029   
47   14022                    0.003483                  0.000048   

    Deuteron Hydrogen Rate cc/s  Helion Hydrogen Rate cc/s  \
0                      0.000284               2.141938e-05   
1                      0.000186               0.000000e+00   
2                      0.000184               7.982863e-06   
3                      0.000111               4.977349e-06   
4                      0.000114               0.000000e+00   
5                      0.000098               1.590101e-06   
6                      0.000124               0.000000e+00   
7                      0.000072               1.886777e-05   
8                      0.000261               2.296519e-06   
9                      0.000273               2.837

In [120]:
#Appending the values to Polymer df
Pol_rates_df['Cell #'] = n_df['cell num']
Pol_rates_df['Neutron Polymer Rate g/s'] = Pol_rate_n
Pol_rates_df['Alpha Polymer Rate g/s'] = Pol_rate_a
Pol_rates_df['Deuteron Polymer Rate g/s'] = Pol_rate_d
Pol_rates_df['Helion Polymer Rate g/s'] = Pol_rate_h
Pol_rates_df['Triton Polymer Rate g/s'] = Pol_rate_t
Pol_rates_df['Proton Polymer Rate g/s'] = Pol_rate_p
Pol_rates_df['Photon Polymer Rate g/s'] = Pol_rate_ph
Pol_rates_df['Pion + Polymer Rate g/s'] = Pol_rate_pi

Pol_rates_df

Cell #  Neutron Polymer Rate g/s  Alpha Polymer Rate g/s  \
0       13                  0.000119            1.208410e-06   
1       14                  0.000106            1.002540e-06   
2    13000                  0.000083            1.132203e-06   
3    14000                  0.000071            1.091373e-06   
4    13001                  0.000051            7.399807e-07   
5    14001                  0.000044            6.913294e-07   
6    13002                  0.000028            3.281253e-07   
7    14002                  0.000025            3.638952e-07   
8    13003                  0.000140            1.658807e-06   
9    14003                  0.000125            1.254824e-06   
10   13004                  0.000090            1.270819e-06   
11   14004                  0.000077            1.113053e-06   
12   13005                  0.000056            7.125779e-07   
13   14005                  0.000047            6.321539e-07   
14   13006                  0.000033            3.936523e-07   
15   14006                  0.000029            5.272780e-07   
16   13007                  0.000139            1.532095e-06   
17   14007                  0.000124            9.046632e-07   
18   13008                  0.000088            1.002858e-06   
19   14008                  0.000075            8.736000e-07   
20   13009                  0.000055            1.091717e-06   
21   14009                  0.000047            6.438010e-07   
22   13010                  0.000034            5.590246e-07   
23   14010                  0.000029            4.964977e-07   
24   13011                  0.000137            1.659921e-06   
25   14011                  0.000125            1.073955e-06   
26   13012                  0.000089            9.861578e-07   
27   14012                  0.000078            1.052154e-06   
28   13013                  0.000058            6.977231e-07   
29   14013                  0.000049            7.685543e-07   
30   13014                  0.000036            3.734980e-07   
31   14014                  0.000032            4.321986e-07   
32   13015                  0.000085            6.499031e-07   
33   14015                  0.000080            5.057031e-07   
34   13016                  0.000060            7.927930e-07   
35   14016                  0.000054            6.170641e-07   
36   13017                  0.000040            6.313574e-07   
37   14017                  0.000035            4.124395e-07   
38   13018                  0.000026            2.343851e-07   
39   14018                  0.000022            2.912186e-07   
40   13019                  0.000048            3.649420e-07   
41   14019                  0.000046            3.679090e-07   
42   13020                  0.000038            2.061536e-07   
43   14020                  0.000035            2.965398e-07   
44   13021                  0.000027            2.976175e-07   
45   14021                  0.000024            2.511441e-07   
46   13022                  0.000016            1.230912e-07   
47   14022                  0.000015            2.071516e-07   

    Deuteron Polymer Rate g/s  Helion Polymer Rate g/s  \
0                1.211813e-06             9.153048e-08   
1                7.945949e-07             0.000000e+00   
2                7.884014e-07             3.411282e-08   
3                4.739348e-07             2.126948e-08   
4                4.852859e-07             0.000000e+00   
5                4.170269e-07             6.794908e-09   
6                5.290625e-07             0.000000e+00   
7                3.076028e-07             8.062680e-08   
8                1.115512e-06             9.813615e-09   
9                1.166291e-06             1.212466e-07   
10               7.557191e-07             2.383876e-08   
11               6.404104e-07             0.000000e+00   
12               2.469405e-07             2.039778e-09   
13               5.139248e-07             0.000000e+00 

## Total Rates for Moderator 
Polymer and Hydrogen production over the whole moderator

In [121]:
#Total raw energy
n_raw_E = n_df['raw_energy_MeV'].sum()
a_raw_E = a_df['raw_energy_MeV'].sum()
d_raw_E = d_df['raw_energy_MeV'].sum()
h_raw_E = h_df['raw_energy_MeV'].sum()
t_raw_E = t_df['raw_energy_MeV'].sum()
p_raw_E = p_df['raw_energy_MeV'].sum()
ph_raw_E =ph_df['raw_energy_MeV'].sum()
pi_raw_E = pi_df['raw_energy_MeV'].sum()
Tot_raw_E = n_raw_E + a_raw_E + d_raw_E + h_raw_E + t_raw_E + p_raw_E + ph_raw_E + pi_raw_E
print('Total Raw Energy', Tot_raw_E, 'MeV')

Total Raw Energy 0.39303443434632307 MeV


In [122]:
#Total Mass and Volume of CH4
Tot_Mass = n_df['mass g'].sum()
print('Total Mass of CH4', Tot_Mass, 'g')

Tot_Vol = n_df['volume cc'].sum()
print('Total Volume of CH4', Tot_Vol, 'cc')


Total Mass of CH4 209.27578266 g
Total Volume of CH4 474.54825999999997 cc


In [123]:
#Total Energy Deposited per g and Dose Rate
E_dep_Tot = (Tot_raw_E/Tot_Mass)*pps
dose_rate_Tot = MeV_per_g_per_sec_to_MGy_per_sec(E_dep_Tot)
print('Absorbed Dose Rate for Whole Moderator', dose_rate_Tot, 'MGy/s')

Absorbed Dose Rate for Whole Moderator 0.00030090051606403677 MGy/s


In [124]:
#Total H and Polymer rates
H_rate_Tot = vol_of_H_per_s(moles_H_per_MGyg, Tot_Vol, dose_rate_Tot, 0.43)
print('Total Hydrogen Production Rate', H_rate_Tot, 'cc/s')
Pol_rate_Tot = rate_of_pol_form(0.22e-6, Mr_20, dose_rate_Tot, Tot_Mass)
print('Total Polymer Production Rate', Pol_rate_Tot, 'g/s')

Total Hydrogen Production Rate 0.9077447114359747 cc/s
Total Polymer Production Rate 0.0038790253657293105 g/s


## Dwell Times
- The picture in the ppt/online is 1/4 geom - im assuming back left?
- Data for estimates taken from "current TS1, split methane forOT" powerpoint - assigned a time to each section as shown on third slide.

**Note: On the dwell times ppt - the image only shows >4s so there are some gaps, the image on the stfc site includes <1s but doesnt show back pic so have to guesstimate**

**Note 2: Average dwell time of 2.6s at outlet taken from powerpoint by Nathan O’Donoghue 2018, for the 'fine' simulation**


In [125]:
#Reading dwell times estimates to df
dt_path = 'dwell_times_estimates.txt'
dt_df = Mod_Data_Reader(dt_path)

In [126]:
#Production of H per section
H_vol_n = H_rate_n * dt_df['dwell time s']
H_vol_a = H_rate_a * dt_df['dwell time s']
H_vol_d = H_rate_d * dt_df['dwell time s']
H_vol_h = H_rate_t * dt_df['dwell time s']
H_vol_t = H_rate_h * dt_df['dwell time s']
H_vol_p = H_rate_p * dt_df['dwell time s']
H_vol_ph = H_rate_ph * dt_df['dwell time s']
H_vol_pi = H_rate_pi * dt_df['dwell time s']

In [133]:
#Production of polymer per section
Pol_mass_n = Pol_rate_n * dt_df['dwell time s']
Pol_mass_a = Pol_rate_a * dt_df['dwell time s']
Pol_mass_d = Pol_rate_d * dt_df['dwell time s']
Pol_mass_h = Pol_rate_h * dt_df['dwell time s']
Pol_mass_t = Pol_rate_t * dt_df['dwell time s']
Pol_mass_p = Pol_rate_p * dt_df['dwell time s']
Pol_mass_ph = Pol_rate_ph * dt_df['dwell time s']
Pol_mass_pi = Pol_rate_pi * dt_df['dwell time s']

In [134]:
#Setting up dataframe for H and polymer production

H_vol_df = pd.DataFrame(columns = ['Cell #',
                                            'Neutron Hydrogen Produced cc',
                                            'Alpha Hydrogen Produced cc',
                                            'Deuteron Hydrogen Produced cc',
                                            'Helion Hydrogen Produced cc',
                                            'Triton Hydrogen Produced cc',
                                            'Proton Hydrogen Produced cc',
                                            'Photon Hydrogen Produced cc',
                                            'Pion + Hydrogen Produced cc'
                                            ])

Pol_mass_df = pd.DataFrame(columns = ['Cell #', 
                                              'Neutron Polymer Produced g',
                                              'Alpha Polymer Produced g',
                                              'Deuteron Polymer Produced g',
                                              'Helion Polymer Produced g',
                                              'Triton Polymer Produced g',
                                              'Proton Polymer Produced g',
                                              'Photon Polymer Produced g',
                                              'Pion + Polymer Produced g'
                                              ])

In [137]:
#Appending values to H dataframe
H_vol_df['Cell #'] = n_df['cell num']
H_vol_df['Neutron Hydrogen Produced cc'] = H_vol_n
H_vol_df['Alpha Hydrogen Produced cc'] = H_vol_a
H_vol_df['Deuteron Hydrogen Produced cc'] = H_vol_d
H_vol_df['Helion Hydrogen Produced cc'] = H_vol_h
H_vol_df['Triton Hydrogen Produced cc'] = H_vol_t
H_vol_df['Proton Hydrogen Produced cc'] = H_vol_p
H_vol_df['Photon Hydrogen Produced cc'] = H_vol_ph
H_vol_df['Pion + Hydrogen Produced cc'] = H_vol_pi

#trying to add a row with totals for each column
H_vol_df

Cell #  Neutron Hydrogen Produced cc  Alpha Hydrogen Produced cc  \
0       13                      0.119411                    0.001216   
1       14                      0.106744                    0.001009   
2    13000                      0.081096                    0.001113   
3    14000                      0.070149                    0.001073   
4    13001                      0.057150                    0.000831   
5    14001                      0.049634                    0.000777   
6    13002                      0.044648                    0.000522   
7    14002                      0.039807                    0.000579   
8    13003                      0.154388                    0.001824   
9    14003                      0.137030                    0.001380   
10   13004                      0.122134                    0.001725   
11   14004                      0.104266                    0.001511   
12   13005                      0.073354                    0.000934   
13   14005                      0.061469                    0.000828   
14   13006                      0.046075                    0.000544   
15   14006                      0.039517                    0.000728   
16   13007                      0.068356                    0.000753   
17   14007                      0.061107                    0.000445   
18   13008                      0.034891                    0.000399   
19   14008                      0.029965                    0.000348   
20   13009                      0.015515                    0.000307   
21   14009                      0.013066                    0.000181   
22   13010                      0.016829                    0.000275   
23   14010                      0.014440                    0.000244   
24   13011                      0.067227                    0.000816   
25   14011                      0.061452                    0.000528   
26   13012                      0.035339                    0.000392   
27   14012                      0.030969                    0.000419   
28   13013                      0.016174                    0.000196   
29   14013                      0.013849                    0.000216   
30   13014                      0.017782                    0.000184   
31   14014                      0.015542                    0.000212   
32   13015                      0.093535                    0.000715   
33   14015                      0.087628                    0.000556   
34   13016                      0.081164                    0.001076   
35   14016                      0.072826                    0.000838   
36   13017                      0.052599                    0.000827   
37   14017                      0.046357                    0.000540   
38   13018                      0.035311                    0.000324   
39   14018                      0.031032                    0.000402   
40   13019                      0.048461                    0.000367   
41   14019                      0.046631                    0.000370   
42   13020                      0.037637                    0.000203   
43   14020                      0.034593                    0.000291   
44   13021                      0.029959                    0.000334   
45   14021                      0.027325                    0.000282   
46   13022                      0.025925                    0.000196   
47   14022                      0.023685                    0.000330   

    Deuteron Hydrogen Produced cc  Helion Hydrogen Produced cc  \
0                        0.001219                 2.746422e-04   
1                        0.000800                 9.014956e-05   
2                        0.000775                 5.585017e-05   
3                        0.000466                 0.000000e+00   
4                        0.000545                 0.000000e+00   
5                        0.000468                 1.141270e-04   
6            

In [143]:
#H totals for each particle
H_vol_df.drop('Cell #', axis=1).sum(axis = 0, skipna = True) 

Neutron Hydrogen Produced cc     2.594043
Alpha Hydrogen Produced cc       0.030157
Deuteron Hydrogen Produced cc    0.022556
Helion Hydrogen Produced cc      0.002151
Triton Hydrogen Produced cc      0.001131
Proton Hydrogen Produced cc      0.339308
Photon Hydrogen Produced cc      0.379617
Pion + Hydrogen Produced cc      0.018981
dtype: float64

In [145]:
#Appending values to polymer dataframe
Pol_mass_df['Cell #'] = n_df['cell num']
Pol_mass_df['Neutron Polymer Produced g'] = Pol_mass_n
Pol_mass_df['Alpha Polymer Produced g'] = Pol_mass_a
Pol_mass_df['Deuteron Polymer Produced g'] = Pol_mass_d
Pol_mass_df['Helion Polymer Produced g'] = Pol_mass_h
Pol_mass_df['Triton Polymer Produced g'] = Pol_mass_t
Pol_mass_df['Proton Polymer Produced g'] = Pol_mass_p
Pol_mass_df['Photon Polymer Produced g'] = Pol_mass_ph
Pol_mass_df['Pion + Polymer Produced g'] = Pol_mass_pi

Pol_mass_df

Cell #  Neutron Polymer Produced g  Alpha Polymer Produced g  \
0       13                    0.000510              5.196162e-06   
1       14                    0.000456              4.310923e-06   
2    13000                    0.000347              4.755254e-06   
3    14000                    0.000300              4.583767e-06   
4    13001                    0.000244              3.551907e-06   
5    14001                    0.000212              3.318381e-06   
6    13002                    0.000191              2.231252e-06   
7    14002                    0.000170              2.474488e-06   
8    13003                    0.000660              7.796391e-06   
9    14003                    0.000586              5.897672e-06   
10   13004                    0.000522              7.370750e-06   
11   14004                    0.000446              6.455705e-06   
12   13005                    0.000313              3.990436e-06   
13   14005                    0.000263              3.540062e-06   
14   13006                    0.000197              2.322548e-06   
15   14006                    0.000169              3.110940e-06   
16   13007                    0.000292              3.217399e-06   
17   14007                    0.000261              1.899793e-06   
18   13008                    0.000149              1.704859e-06   
19   14008                    0.000128              1.485120e-06   
20   13009                    0.000066              1.310060e-06   
21   14009                    0.000056              7.725612e-07   
22   13010                    0.000072              1.173952e-06   
23   14010                    0.000062              1.042645e-06   
24   13011                    0.000287              3.485834e-06   
25   14011                    0.000263              2.255305e-06   
26   13012                    0.000151              1.676468e-06   
27   14012                    0.000132              1.788661e-06   
28   13013                    0.000069              8.372677e-07   
29   14013                    0.000059              9.222652e-07   
30   13014                    0.000076              7.843457e-07   
31   14014                    0.000066              9.076171e-07   
32   13015                    0.000400              3.054544e-06   
33   14015                    0.000374              2.376805e-06   
34   13016                    0.000347              4.598199e-06   
35   14016                    0.000311              3.578972e-06   
36   13017                    0.000225              3.535602e-06   
37   14017                    0.000198              2.309661e-06   
38   13018                    0.000151              1.382872e-06   
39   14018                    0.000133              1.718189e-06   
40   13019                    0.000207              1.569251e-06   
41   14019                    0.000199              1.582009e-06   
42   13020                    0.000161              8.658451e-07   
43   14020                    0.000148              1.245467e-06   
44   13021                    0.000128              1.428564e-06   
45   14021                    0.000117              1.205492e-06   
46   13022                    0.000111              8.370200e-07   
47   14022                    0.000101              1.408631e-06   

    Deuteron Polymer Produced g  Helion Polymer Produced g  \
0                  5.210797e-06               3.935811e-07   
1                  3.416758e-06               0.000000e+00   
2                  3.311286e-06               1.432738e-07   
3                  1.990526e-06               8.933183e-08   
4                  2.329372e-06               0.000000e+00   
5                  2.001729e-06               3.261556e-08   
6                  3.597625e-06               0.000000e+00   
7                  2.091699e-06               5.482623e-07   
8                  5.242905e-06               4.612399e-08   
9                  5.481569e-06               5.698

In [146]:
#Polymer totals for each particle
Pol_mass_df.drop('Cell #', axis=1).sum(axis = 0, skipna = True)

Neutron Polymer Produced g     0.011085
Alpha Polymer Produced g       0.000129
Deuteron Polymer Produced g    0.000096
Helion Polymer Produced g      0.000005
Triton Polymer Produced g      0.000009
Proton Polymer Produced g      0.001450
Photon Polymer Produced g      0.001622
Pion + Polymer Produced g      0.000081
dtype: float64

In [148]:
#Averages for the moderator as a whole - for one cycle
H_vol_avg = H_rate_Tot * 2.6
print('Average Hydrogen gas produced in the whole moderator', H_vol_avg, 'cc')
Pol_mass_avg = Pol_rate_Tot * 2.6
print('Average polymer produced in the whole moderator', Pol_mass_avg, 'g')

Average Hydrogen gas produced in the whole moderator 2.360136249733534 cc
Average polymer produced in the whole moderator 0.010085465950896207 g
